# **K-Nearest Neighbors (KNN) Class Implementation in Python**

In [1]:
# required libraries
from sklearn.datasets import load_iris
import torch

In [27]:
class KNearestNeighbor():
  def __init__(self,X_train,y_train,X_test,k = 5):
    self.__X_train = X_train                     # training dataset
    self.__X_test = X_test                     # data/point to predict class/es
    self.__trainN = self.__X_train.shape[0]    # number of trainig samples
    self.__d = self.__X_train.shape[1]         # number of features
    self.__testN = self.__X_test.shape[0]      # number of testing samples
    self.__k = k                               # neighbors to consider


  @classmethod
  def train_test_split(cls,X,y,split_ratio,k = 5):
    # select random indices
    indices = torch.randperm(X.size(0))

    train_N = int(split_ratio * X.size(0))

    train_indices = indices[:train_N]
    test_indices = indices[train_N:]

    X_train = X[train_indices]
    y_train = y[train_indices]

    X_test = X[test_indices]
    y_test = X[test_indices]

    return cls(X_train,y_train,X_test,k)

  def euclidean_distance(self,point1,point2):
      try:
        n = point1.shape(0)
        distance = 0
        for i in range(n):
          distance += point1[i] - point2[i]
        return torch.sqrt(distance)
      except Exception as e:                            # Raise Exception if point1 and point2 are of different dimension
        print(f'Exception: {type(e).__name__} - {e}')

  def classify(self):
    distances = []
    for i in range(self.__testN):
      test = self.__X_test[i]
      for j in range(self.__trainN):
        distances = [(self.euclidean_distance(train,test)) for train in enumerate(self.__X_train)]
    return distances






# **Evaluating KNN Class on Iris Dataset**

In [28]:
iris = load_iris()
X  = torch.from_numpy(iris.data)
y = torch.from_numpy(iris.target)

obj = KNearestNeighbor.train_test_split(X,y,0.8)
obj.classify()

Streaming output truncated to the last 5000 lines.
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shape'
Exception: AttributeError - 'tuple' object has no attribute 'shap

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]